In [0]:
from datetime import datetime

current_date = datetime.now()
year = current_date.strftime("%Y")
month = current_date.strftime("%m")
day = current_date.strftime("%d")

party = spark.readStream \
    .format("csv") \
    .option("header", "true") \
    .load(f"s3://dms-my-target-postgres-rds/public/party/{year}/{month}/{day}/")

In [0]:
party_cleaned.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option(
        "checkpointLocation",
        "s3://dms-my-target-postgres-rds/processed/party/party_raw/_checkpoint"
    ) \
    .start("s3://dms-my-target-postgres-rds/processed/party/party_raw")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2042006140274726>, line 8
      1 party_cleaned.writeStream \
      2     .format("delta") \
      3     .outputMode("append") \
      4     .option(
      5         "checkpointLocation",
      6         "s3://dms-my-target-postgres-rds/processed/party/party_raw/_checkpoint"
      7     ) \
----> 8     .start("s3://dms-my-target-postgres-rds/processed/party/party_raw")

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/streaming/readwriter.py:713, in DataStreamWriter.start(self, path, format, outputMode, partitionBy, queryName, **options)
    704 def start(
    705     self,
    706     path: Optional[str] = None,
   (...)
    711     **options: "OptionalPrimitiveType",
    712 ) -> "StreamingQuery":
--> 713     return self._start_internal(
    714         path=path,
    715         tableName=None,
 

In [0]:
from pyspark.sql.functions import to_timestamp, date_format, col, row_number, expr
from pyspark.sql.window import Window

# Convert event_time_source to timestamp and format as dd/MM/yyyy HH:mm:ss
party_with_ts = party_cleaned.withColumn(
    "event_time_source_ts",
    to_timestamp("event_time_source")
).withWatermark("event_time_source_ts", "10 minutes")

party_formatted = party_with_ts.withColumn(
    "event_time_source_formatted",
    date_format(col("event_time_source_ts"), "dd/MM/yyyy HH:mm:ss")
)

# Window to get latest event_time_source per party_id in each micro-batch
window_spec = Window.partitionBy("party_id").orderBy(col("event_time_source_ts").desc())

party_latest = (
    party_formatted.withColumn(
        "rn", row_number().over(window_spec)
    )
    .filter(col("rn") == 1)
    .drop("rn")
    .drop("event_time_source")
)

#display(party_latest)

In [0]:
# Read from Delta table (not cloudFiles) if the source is Delta
party_latest = (
    spark.readStream
    .format("delta")
    .load("s3://dms-my-target-postgres-rds/processed/party/party_raw")
)

# Rename column
party_gold_df = party_latest.withColumnRenamed(
    "event_time_source_formatted", "effective_date"
)

# Write stream to Delta table
party_gold_df.writeStream (
    .format("delta")
    .outputMode("append")
    .option(
        "checkpointLocation",
        "s3://dms-my-target-postgres-rds/gold/party/party_gold/_checkpoint"
    )
    .start(gold_table_path)
)

# Register gold table as a SQL table if not exists
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS party_gold
    USING DELTA
    LOCATION '{gold_table_path}'
""")

# Register party_gold_df as a temp view for SQL access
party_gold_df.createOrReplaceTempView("party_latest_gold")

  File <command-2042006140274728>, line 15
    .format("delta")
    ^
SyntaxError: invalid syntax


In [0]:
%sql
--select * from party_latest_gold

In [0]:
from delta.tables import DeltaTable

deltaTable = DeltaTable.forPath(spark, gold_table_path)

deltaTable.alias("gold").merge(
    source=spark.table("party_latest_gold").alias("latest"),
    condition="gold.party_id = latest.party_id",
    whenMatchedUpdate={"party_id": "latest.party_id", "effective_date": "latest.effective_date", "name": "latest.name", "type": "latest.type"},
    whenMatchedDelete="latest.Op = 'D'",
    whenNotMatchedInsert={"party_id": "latest.party_id", "name": "latest.name", "type": "latest.type", "effective_date": "latest.effective_date"}
)

In [0]:
display(spark.sql("SELECT * FROM party_gold"))

In [0]:
#spark.sql("TRUNCATE TABLE party_gold")